In [2]:
import numpy as np
import collections
import pickle
import os
import argparse
from openbabel import openbabel as ob
from openbabel import pybel
import pandas as pd
import time
import sys

sys.path.append("..")

from analyses import analysis
from schnetpack import Properties
from multiprocessing import Process, Queue
from ase import Atoms
from ase.db import connect

sys.path.append('..')
sys.path.append('../analyses')

import analysis
import datatypes
import qm9_filter_generated
from utility_classes import Molecule, ConnectivityCompressor
from utility_functions import run_threaded, print_atom_bond_ring_stats, update_dict

In [44]:
class Arguments:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

args = Arguments(**{
    'data_path': '../analyses/molecules/generated/v5/nequip/interactions=4/l=5/channels=32/beta=1.0/step=best/',
    'model_path': '../workdirs/v5/nequip/interactions=4/l=5/channels=32/',
    'train_data_path':'../qm9_data/qm9-all.db',
    "valence": [1, 1, 6, 4, 7, 3, 8, 2, 9, 1],
    "filters": ["valence", "disconnected", "unique"],
    "store": "valid_connectivity",
    'threads': 0
 })

In [45]:
# read input file or fuse dictionaries if data_path is a folder
if not os.path.isdir(args.data_path):
    if not os.path.isfile(args.data_path):
        print(
            f"\n\nThe specified data path ({args.data_path}) is neither a file "
            f"nor a directory! Please specify a different data path."
        )
        raise FileNotFoundError
    else:
        with open(args.data_path, "rb") as f:
            res = pickle.load(f)  # read input file
        target_db = os.path.join(
            os.path.dirname(args.data_path), "generated_molecules.db"
        )
else:
    print(f"\n\nFusing .mol_dict files in folder {args.data_path}...")
    mol_files = [f for f in os.listdir(args.data_path) if f.endswith(".mol_dict")]
    if len(mol_files) == 0:
        print(
            f"Could not find any .mol_dict files at {args.data_path}! Please "
            f"specify a different data path!"
        )
        raise FileNotFoundError
    res = {}
    for file in mol_files:
        with open(os.path.join(args.data_path, file), "rb") as f:
            cur_res = pickle.load(f)
            update_dict(res, cur_res)
    res = dict(sorted(res.items()))  # sort dictionary keys
    print(f"...done!")
    target_db = os.path.join(args.data_path, "generated_molecules.db")

# compute array with valence of provided atom types
max_type = max(args.valence[::2])
valence = np.zeros(max_type + 1, dtype=int)
valence[args.valence[::2]] = args.valence[1::2]

# print the chosen settings
valence_str = ""
for i in range(max_type + 1):
    if valence[i] > 0:
        valence_str += f"type {i}: {valence[i]}, "
filters = []
if "valence" in args.filters:
    filters += ["valency"]
if "disconnected" in args.filters:
    filters += ["connectedness"]
if "unique" in args.filters:
    filters += ["uniqueness"]
if len(filters) >= 3:
    edit = ", "
else:
    edit = " "
for i in range(len(filters) - 1):
    filters[i] = filters[i] + edit
if len(filters) >= 2:
    filters = filters[:-1] + ["and "] + filters[-1:]
string = "".join(filters)
print(f"\n\n1. Filtering molecules according to {string}...")
print(f"\nTarget valence:\n{valence_str[:-2]}\n")

# initial setup of array for statistics and some counters
n_generated = 0
n_valid_mol = 0
n_non_unique = 0
stat_heads = [
    "n_atoms",
    "id",
    "valid_mol",
    "valid_atom",
    "duplicating",
    "n_duplicates",
    "known",
    "equals",
    "C",
    "N",
    "O",
    "F",
    "H",
    "H1C",
    "H1N",
    "H1O",
    "C1C",
    "C2C",
    "C3C",
    "C1N",
    "C2N",
    "C3N",
    "C1O",
    "C2O",
    "C1F",
    "N1N",
    "N2N",
    "N1O",
    "N2O",
    "N1F",
    "O1O",
    "O1F",
    "R3",
    "R4",
    "R5",
    "R6",
    "R7",
    "R8",
    "R>8",
]
stats = np.empty((len(stat_heads), 0))
all_mols = []
connectivity_compressor = ConnectivityCompressor()

# construct connectivity matrix and fingerprints for filtering
start_time = time.time()
for n_atoms in res:
    if not isinstance(n_atoms, int) or n_atoms == 0:
        continue

    prog_str = lambda x: f"Checking {x} for molecules of length {n_atoms}"
    work_str = "valence" if "valence" in args.filters else "dictionary"

    d = res[n_atoms]
    all_pos = d[Properties.R]
    all_numbers = d[Properties.Z]
    n_mols = len(all_pos)

    # check valency
    if args.threads <= 0:
        results = qm9_filter_generated.check_valency(
            all_pos,
            all_numbers,
            valence,
            "valence" in args.filters,
            False,
            prog_str(work_str),
        )
    else:
        results = {
            "connectivity": np.zeros((n_mols, n_atoms, n_atoms)),
            "mols": [None for _ in range(n_mols)],
            "valid_mol": np.ones(n_mols, dtype=bool),
            "valid_atom": np.ones(n_mols, dtype=bool),
        }
        results = run_threaded(
            qm9_filter_generated.check_valency,
            {"positions": all_pos, "numbers": all_numbers},
            {
                "valence": valence,
                "filter_by_valency": "valence" in args.filters,
                "picklable_mols": True,
                "prog_str": prog_str(work_str),
            },
            results,
            n_threads=args.threads,
            exclusive_kwargs={"print_file": False},
        )
    connectivity = results["connectivity"]
    mols = results["mols"]
    valid_mol = results["valid_mol"]
    valid_atom = results["valid_atom"]

    # detect molecules with disconnected parts if desired
    if "disconnected" in args.filters:
        if args.threads <= 0:
            valid_mol = qm9_filter_generated.remove_disconnected(connectivity, valid_mol)["valid_mol"]
        else:
            results = {"valid_mol": valid_mol}
            run_threaded(
                qm9_filter_generated.remove_disconnected,
                {"connectivity_batch": connectivity, "valid_mol": valid_mol},
                {},
                results,
                n_threads=args.threads,
            )
            valid_mol = results["valid_mol"]

    # identify molecules with identical fingerprints
    if args.threads <= 0:
        still_valid, duplicating, duplicate_count = qm9_filter_generated.filter_unique(
            mols, valid_mol, use_bits=False
        )
    else:
        still_valid, duplicating, duplicate_count = qm9_filter_generated.filter_unique_threaded(
            mols,
            valid_mol,
            n_threads=args.threads,
            n_mols_per_thread=5,
            print_file=False,
            prog_str=prog_str("uniqueness"),
        )
    n_non_unique += np.sum(duplicate_count)
    if "unique" in args.filters:
        valid_mol = still_valid  # remove non-unique from valid if desired

    # store connectivity matrices
    d.update(
        {
            "connectivity": connectivity_compressor.compress_batch(connectivity),
            "valid_mol": valid_mol,
        }
    )

    # collect statistics of generated data
    n_generated += len(valid_mol)
    n_valid_mol += np.sum(valid_mol)
    n_of_types = [np.sum(all_numbers == i, axis=1) for i in [6, 7, 8, 9, 1]]
    stats_new = np.stack(
        (
            np.ones(len(valid_mol)) * n_atoms,  # n_atoms
            np.arange(0, len(valid_mol)),  # id
            valid_mol,  # valid molecules
            valid_atom,  # valid atoms (atoms with correct valence)
            duplicating,  # id of duplicated molecule
            duplicate_count,  # number of duplicates
            -np.ones(len(valid_mol)),  # known
            -np.ones(len(valid_mol)),  # equals
            *n_of_types,  # n_atoms per type
            *np.zeros((19, len(valid_mol))),  # n_bonds per type pairs
            *np.zeros((7, len(valid_mol))),  # ring counts for 3-8 & >8
        ),
        axis=0,
    )
    stats = np.hstack((stats, stats_new))
    all_mols += mols

end_time = time.time() - start_time
m, s = divmod(end_time, 60)
h, m = divmod(m, 60)
h, m, s = int(h), int(m), int(s)
print(f"Needed {h:d}h{m:02d}m{s:02d}s.")

if args.threads <= 0:
    results = qm9_filter_generated.collect_bond_and_ring_stats(all_mols, stats.T, stat_heads)
else:
    results = {"stats": stats.T}
    run_threaded(
        qm9_filter_generated.collect_bond_and_ring_stats,
        {"mols": all_mols, "stats": stats.T},
        {"stat_heads": stat_heads},
        results=results,
        n_threads=args.threads,
    )
stats = results["stats"].T

# store statistics
res.update(
    {
        "n_generated": n_generated,
        "n_valid_mol": n_valid_mol,
        "stats": stats,
        "stat_heads": stat_heads,
    }
)

print(
    f"Number of generated molecules: {n_generated}\n"
    f"Number of duplicate molecules: {n_non_unique}"
)
if "unique" in args.filters:
    print(f"Number of unique and valid molecules: {n_valid_mol}")
else:
    print(f"Number of valid molecules (including duplicates): {n_valid_mol}")

# filter molecules which were seen during training
if args.model_path is not None:
    stats = qm9_filter_generated.filter_new(
        all_mols,
        stats,
        stat_heads,
        args.model_path,
        args.train_data_path,
        print_file=False,
        n_threads=args.threads,
    )
    res.update({"stats": stats})

# shrink results dictionary (remove invalid attempts, known molecules and
# connectivity matrices if desired)
if args.store != "all":
    shrunk_res = {}
    shrunk_stats = np.empty((len(stats), 0))
    i = 0
    for key in res:
        if isinstance(key, str):
            shrunk_res[key] = res[key]
            continue
        if key == 0:
            continue
        d = res[key]
        start = i
        end = i + len(d["valid_mol"])
        idcs = np.where(d["valid_mol"])[0]
        if len(idcs) < 1:
            i = end
            continue
        # shrink stats
        idx_id = stat_heads.index("id")
        idx_known = stat_heads.index("known")
        new_stats = stats[:, start:end]
        if "new" in args.store and args.model_path is not None:
            idcs = idcs[np.where(new_stats[idx_known, idcs] == 0)[0]]
        new_stats = new_stats[:, idcs]
        new_stats[idx_id] = np.arange(len(new_stats[idx_id]))  # adjust ids
        shrunk_stats = np.hstack((shrunk_stats, new_stats))
        # shrink positions and atomic numbers
        shrunk_res[key] = {
            Properties.R: d[Properties.R][idcs],
            Properties.Z: d[Properties.Z][idcs],
        }
        # store connectivity matrices if desired
        if "connectivity" in args.store:
            shrunk_res[key].update(
                {"connectivity": [d["connectivity"][k] for k in idcs]}
            )
        i = end

    shrunk_res["stats"] = shrunk_stats
    res = shrunk_res




Fusing .mol_dict files in folder ../analyses/molecules/generated/v5/nequip/interactions=4/l=5/channels=32/beta=1.0/step=best/...
...done!


1. Filtering molecules according to valency, connectedness, and uniqueness...

Target valence:
type 1: 1, type 6: 4, type 7: 3, type 8: 2, type 9: 1

Needed 0h00m00s. for molecules of length 31 (100.00%)
Number of generated molecules: 64
Number of duplicate molecules: 0
Number of unique and valid molecules: 12


2. Checking which molecules are new...
Using data base at ../qm9_data/qm9-all.db...


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBond


Computing fingerprints of training data...


*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders



*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBond

...

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBond

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBond

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBond

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBond

error getting idx=133885
error getting idx=133886
error getting idx=133887
error getting idx=133888
error getting idx=133889
error getting idx=133890
error getting idx=133891
error getting idx=133892
error getting idx=133893
error getting idx=133894
error getting idx=133895
error getting idx=133896
error getting idx=133897
error getting idx=133898
error getting idx=133899
error getting idx=133900
error getting idx=133901
error getting idx=133902
error getting idx=133903
error getting idx=133904
error getting idx=133905
error getting idx=133906
error getting idx=133907
error getting idx=133908
error getting idx=133909
error getting idx=133910
error getting idx=133911
error getting idx=133912
error getting idx=133913
error getting idx=133914
error getting idx=133915
error getting idx=133916
error getting idx=133917
error getting idx=133918
error getting idx=133919
...133920 fingerprints computed in 0h01m08s!

Comparing fingerprints...
.00%K... needed 0h00m00s.
Number of new molecules: 5


In [ ]:
methane = analysis.construct_molecule('CH3')[0]
valence = np.array([0, 1, 0, 0, 0, 0, 4, 3, 2, 1])

In [5]:
qm9_filter_generated.check_valency(np.asarray([methane.positions]), np.asarray([list(methane.numbers)]), valence)

{'mols': [<utility_classes.Molecule at 0x107957850>],
 'connectivity': array([[[0., 1., 1., 1.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.]]]),
 'valid_mol': array([False]),
 'valid_atom': array([0.75])}

In [3]:
config, best_state, _, metrics_for_best_state = analysis.load_from_workdir('../workdirs/v5/mace/interactions=4/l=5/channels=32')

In [58]:
# store gathered statistics in metrics dataframe
with open('../analyses/molecules/generated/v5/nequip/interactions=4/l=5/channels=32/beta=1.0/step=best/generated_molecules_statistics.pkl', 'rb') as f:
    stats_df = pickle.load(f)


In [60]:
stats_df.keys()

dict_keys(['train_eval_final', 'val_eval_final', 'test_eval_final', 'generated_stats_overall', 'generated_stats'])

In [62]:
stats_df['generated_stats']

,n_atoms,id,valid_mol,valid_atom,duplicating,n_duplicates,known,equals,C,N,...,N1F,O1O,O1F,R3,R4,R5,R6,R7,R8,R>8
0,4.0,0.0,1.0,1.0,-1.0,0.0,1.0,3.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6.0,0.0,1.0,1.0,-1.0,0.0,1.0,25.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6.0,1.0,1.0,1.0,-1.0,0.0,1.0,22.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7.0,0.0,1.0,1.0,-1.0,0.0,0.0,-1.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8.0,0.0,1.0,1.0,-1.0,0.0,1.0,489.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,9.0,0.0,1.0,1.0,-1.0,0.0,1.0,14562.0,7.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,10.0,0.0,1.0,1.0,-1.0,0.0,0.0,-1.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,11.0,0.0,1.0,1.0,-1.0,0.0,1.0,9999.0,4.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,14.0,0.0,1.0,1.0,-1.0,0.0,2.0,51463.0,5.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
9,14.0,1.0,1.0,1.0,-1.0,0.0,0.0,-1.0,5.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
